## Ejercicio 2
Utilizar Simpy para simular una sección del ejercicio 1 a elección

In [ ]:
from generators import LXM, GCL
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import simpy
import scipy

Parametros iniciales

In [ ]:
lambdaArribos = 9
lambdaServicio = 10
cantidadArribos = 20000
generador = LXM()

Vamos a necesitar generar valores exponenciales, los obtenemos con la transformada inversa.

In [ ]:
def exponencial(lambdaArgumento, generador):
    uniforme = generador.generar()
    return - np.log(1 - uniforme) / lambdaArgumento

Definimos el modelo del satelite y la solicitud.

In [ ]:
class Satelite:
    def __init__(self, env, servicioDescarga, lambdaArgumento, generador):
        self.env = env
        self.lambdaArgumento = lambdaArgumento
        self.generador = generador
        self.servicioDescarga = servicioDescarga

    def atencion(self, id):
        tiempoAtencion = exponencial(self.lambdaArgumento, self.generador)
        servicio = self.servicioDescarga.request()
        yield servicio
        print("Satelite en uso por la solicitud con id " + str(id))
        N = 1
        largoCola = len(self.servicioDescarga.queue)
        if largoCola != 0:
            N = largoCola
        tiempoServicio = tiempoAtencion / N
        print("El tiempo de atencion obtenido es: " + str(tiempoAtencion)  + ", tiempoServicio: " + str(tiempoServicio))
        print("En la cola hay " + str(len(self.servicioDescarga.queue)))
        yield env.timeout(tiempoServicio)
        self.servicioDescarga.release(servicio)

class Solicitud:
    def __init__(self, env, satelite, id):
        self.env = env
        self.satelite = satelite
        self.id = id

    def tratarse(self):
        return self.satelite.atencion(self.id)

Creamos la funcion que se va a encargar de generar las solicitudes. Estas son nuevos procesos que van a intentar usar el servicio del satelite.

A medida que van llegando de avanza el tiempo del ambiente.

In [ ]:
tiempoEntreArribos = []
tiemposAcumulados = []

def generarArribos(lambdaArgumento, generador, cantidadArribos, env, satelite):
    tiempoAcumulado = 0
    id = 0
    while (id < cantidadArribos):
        tiempo = exponencial(lambdaArgumento, generador)
        nuevaSolicitud = Solicitud(env, satelite, id)
        id = 1 + id
        env.process(nuevaSolicitud.tratarse())
        yield env.timeout(tiempo)

        tiempoAcumulado += tiempo
        tiemposAcumulados.append(tiempoAcumulado)
        tiempoEntreArribos.append(tiempo)

    print("Se crearon un total de " + str(id) + " solicitudes")



Ejecutamos la simulacion

In [ ]:
env = simpy.Environment()
servicioDescarga = simpy.Resource(env, capacity=1)
satelite = Satelite(env,servicioDescarga,lambdaServicio,generador)

env.process(generarArribos(lambdaArribos, generador, cantidadArribos, env, satelite))

env.run()

Vemos los resultados

In [ ]:
sns.kdeplot(tiempoEntreArribos,cut=0)

In [ ]:
plt.step(tiemposAcumulados,range(len(tiemposAcumulados)),where= 'post' ,label='λ=' + str(lambdaArribos))
plt.legend()
plt.show()